## POS tagging using modified Viterbi ALgorithm

#### Date: 11-May-2020

#### Project Description: Enhancement / imporvision of Vanilla Viterbi Algorithm.

#### Operational Steps:
1. Developing plain Vanilla Viterbi Algorithm.

#### Techniques:
2. Implication of Viterbi Modification Technique 1: following are the approach for the same:-
   - Transition probability is considered in case of unknown words.
   - The above approach is modified to consider transition probability weighted by tag occurrence probability in training set.
3. Implication of Viterbi Modification Technique 2: following are the approach for the same:-
   - backoff to a rule based tagger in case of an unknown word.
   - The above technique is modified by using transition probability in approach 2 above if rule based tagger returns default noun tag ('NN').

#### Validation
4. The modified Viterbi algorithms are first tested on sampled test data for comparison.
5. The final algorithm and vanilla Viterbi Algorithm are the tested on full testing data for comparison.


### Data Preparation

In [4]:
#Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time


In [5]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

# checking some tagged data from the data set
print(nltk_data[1:5])

[[('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('conglomerate', 'NOUN'), ('.', '.')], [('A', 'DET'), ('form', 'NOUN'), ('of', 'ADP'), ('asbestos', 'NOUN'), ('once', 'ADV'), ('used', 'VERB'), ('*', 'X'), ('*', 'X'), ('to', 'PRT'), ('make', 'VERB'), ('Kent', 'NOUN'), ('cigarette', 'NOUN'), ('filters', 'NOUN'), ('has', 'VERB'), ('caused', 'VERB'), ('a', 'DET

In [6]:
# splitting the data into training and test with ratio of 95:5
train_set, test_set = train_test_split(nltk_data,train_size = 0.95, test_size = 0.05, random_state = 101)

In [12]:
# creating the list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

# printing the details of the tagged words
print('Train Tagged Words: \t',len(train_tagged_words))
print('Test Tagged words: \t',len(test_tagged_words))

#printing some of the tagged words from tain set.
print('Tagged words from train set: \t',train_tagged_words[1:5])

# printing the list of unique tags in the data set:
tags = {tag for word, tag in train_tagged_words}
print('No of unique tags: \t',len(tags))
print('Unique Tags: \n', tags)

# printing the count of words present:
vocab={word for word, tag in train_tagged_words}
print('count of words in vocab:\t',len(vocab))

Train Tagged Words: 	 95547
Test Tagged words: 	 5129
Tagged words from train set: 	 [('confirmed', 'VERB'), ('the', 'DET'), ('filing', 'NOUN'), ('but', 'CONJ')]
No of unique tags: 	 12
Unique Tags: 
 {'PRON', 'DET', 'X', 'NUM', 'VERB', '.', 'ADP', 'NOUN', 'ADJ', 'PRT', 'CONJ', 'ADV'}
count of words in vocab:	 12100


### POS Tagging algorithm using Hiddne Markov Model (HMM)

Given a sequence of words to be tagged, the task is to assign the most probable tag to the wod. In other words, for every word "w" assign the tag "t" that maximises the likelihood P(t/w)

Since P(t/w) = P(w/t). P(t) / P(w), after ignoring P(w), we have to compute P(w/t) and P(t).

Now:

- P(w/t): is the emission probability of a given word for a given tag. This can be computed based on the fraction of given word for given tag to the total count of that tag, ie: P(w/t) = count(w, t) / count(t).

- P(t): is the probability of tag (also transition probability), and in a tagging task, we assume that a tag will depend only on the previous tag (Markov order 1 assumption). In other words, the probability of say a tag being NN will depend only on the previous tag t(n-1). So for e.g. if t(n-1) is a JJ, then t(n) is likely to be an NN since adjectives often precede a noun (blue coat, tall building etc.).

**ref, HMM definition

### Build the vanilla Viterbi based POS tagger

In [35]:
#function to compute emission probabilities for the give word.
def word_given_tag(word,tag,train_bag = train_tagged_words):
    taglist = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(taglist)
    w_in_tag = [pair[0] for pair in taglist if pair[0] ==word]
    word_count_given_tag = len(w_in_tag)
    return(word_count_given_tag, tag_count)

# fucntion to compute the transition probability of a previous and next tag
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags= [pair[1] for pair in train_bag]
    t1_tags = [tag for tag in tags if tag==t1]
    count_t1 = len(t1_tags)
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index]==t1 and tags[index+1]==t2]
    count_t2_given_t1 = len(t2_given_t1)
    return (count_t2_given_t1,count_t1)

## dummy Check
t2_given_t1('NOUN','DET')


(5284, 8281)

In [44]:
# creating ixj transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j,t2 in enumerate(list(tags)):
        tags_matrix[i,j] = t2_given_t1(t2,t1)[0]/t2_given_t1(t2,t1)[1]
# tags_matrix
# converting the matrix to DF for readibility ease.
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index = list(tags))
tags_df

,PRON,DET,X,NUM,VERB,.,ADP,NOUN,ADJ,PRT,CONJ,ADV
PRON,0.007657,0.009954,0.089969,0.006508,0.485452,0.040965,0.022971,0.210949,0.073124,0.013017,0.005360,0.034074
DET,0.003744,0.005676,0.045405,0.022220,0.039850,0.017993,0.009540,0.638087,0.204323,0.000242,0.000483,0.012438
X,0.055538,0.054742,0.076384,0.002864,0.203851,0.163590,0.142584,0.062381,0.017187,0.185232,0.010662,0.024984
NUM,0.001489,0.003276,0.210542,0.184932,0.018761,0.117332,0.036033,0.350208,0.034247,0.026504,0.013699,0.002978
VERB,0.035786,0.134392,0.217506,0.022851,0.169249,0.034934,0.092022,0.110070,0.064988,0.030674,0.005577,0.081952
.,0.066349,0.173335,0.026971,0.081003,0.089095,0.093320,0.091342,0.222242,0.043963,0.002427,0.057538,0.052324
ADP,0.070031,0.324709,0.034427,0.062226,0.008340,0.039025,0.016893,0.320967,0.107024,0.001390,0.000962,0.014006
NOUN,0.004607,0.012942,0.029175,0.009542,0.147667,0.240604,0.176514,0.263564,0.012248,0.043397,0.042666,0.017074
ADJ,0.000330,0.004943,0.021091,0.021256,0.011699,0.063931,0.078267,0.699621,0.066403,0.010710,0.016971,0.004778
PRT,0.017792,0.097858,0.013509,0.056672,0.405272,0.043822,0.020099,0.247776,0.083031,0.001647,0.002306,0.010214


# Viterbi Algorithm

The steps are as follows:

   -  Given a sequence of words iterate through the sequence for each word (starting from first word in sequence) 
   -  calculate the product of emission probabilties and transition probabilties for all possible tags.
   -  assign the tag which has maximum probability and move to the next word in sequence



In [61]:
# Vanilla Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [70]:
# testin Viterbi algorithm
random.seed(1234)
# random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]
#list of sents
test_run = [test_set[i] for i in rndom]
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [73]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time lapsed: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Time lapsed:  36.299710512161255
Vanilla Viterbi Algorithm Accuracy:  88.49557522123894


In [82]:
# checking incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('Contra', 'PRON'), ('Contra', 'NOUN')),
 (('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'PRON'), ('Honduras', 'NOUN')),
 (('Sandinista', 'PRON'), ('Sandinista', 'NOUN')),
 (('offensive', 'PRON'), ('offensive', 'NOUN')),
 (('rebel', 'PRON'), ('rebel', 'NOUN')),
 (('forces', 'VERB'), ('forces', 'NOUN')),
 (('Bucking', 'PRON'), ('Bucking', 'VERB')),
 (('drew', 'PRON'), ('drew', 'VERB')),
 (('Eveready', 'PRON'), ('Eveready', 'NOUN')),
 (('*T*-252', 'PRON'), ('*T*-252', 'X')),
 (('complaining', 'PRON'), ('complaining', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

### Solve the problem of unknown words

### Viterbi Modification-Technique I

- First solution for unknown words: assign based on transition probabilities only in case of unknown words as emission probability for unknown word is zero.


In [115]:
# transition probability of tags when emission probability is zero (in case of unknown words)

def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = [] 
        p_transition =[] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
        else:
            state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [116]:
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start
print("time lapsed: ", difference)
check = [i for i,j in zip(tagged_seq, test_run_base) if i==j]
accuracy = len(check)/len(tagged_seq)
print("Viterbi_1 Accuracy: ", accuracy*100)

time lapsed:  38.03657555580139
Viterbi_1 Accuracy:  94.69026548672566


### Adding Tag occurance probability weights: 
applying weights based on the probability of tag occurance to the transition probabilities of tags and then use the resulting probability for predicting unknown words.

In [120]:
tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in tags:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('PRON', 0.0273373313657153),
 ('DET', 0.08666938784053921),
 ('X', 0.06576867928872701),
 ('NUM', 0.035145007169246546),
 ('VERB', 0.1351167488251855),
 ('.', 0.11641391147812072),
 ('ADP', 0.0978889970381069),
 ('NOUN', 0.2862674913916711),
 ('ADJ', 0.06351847781719991),
 ('PRT', 0.03176447193527793),
 ('CONJ', 0.02251248076862696),
 ('ADV', 0.031597015081582885)]

In [166]:
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = [] 
        p_transition =[]  
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                 
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [168]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start
print("Time lapsed: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time lapsed:  37.99132752418518
Modified Viterbi_1 Accuracy:  95.57522123893806


- Using the weighted transition probabilites we have improved the accuracy

In [178]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_1 as compared to vanilla Viterbi Algorithm:

    Contra:correctly tagged as NOUN
    Honduras:correctly tagged as NOUN
    complaining: correctly tagged as VERB
    Bucking: correctly tagged as VERB

### Viterbi Modification-Technique II
### second solution for unknown words:  backoff to rule based tagger
   POS tag 'X' can be easily encapsulated in regex rule, so extraction is only based on ruled based tagged.

In [188]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)



In [201]:
# Modification in Viterbi Algorithm : Backoff to rule based tagger 
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1]
        else:
            if state_max != 'X':
                state_max = T[p.index(pmax)]                
        state.append(state_max)
    return list(zip(words, state))

In [202]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start
print("Time lapsed: ", difference)
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Time lapsed:  39.213961362838745
Modified Viterbi_2 Accuracy:  97.34513274336283


In [205]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('drew', 'NOUN'), ('drew', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]


The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

    Contra:correctly tagged as NOUN
    Honduras:correctly tagged as NOUN
    complaining: correctly tagged as VERB
    Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1

    Sandinista: correctly tagged as NOUN
    Eveready: correctly tagged as NOUN
    *T*-252: correctly tagged as 'X'

further modification in Viterb_2: We know that the rule based tagger assigns 'NOUN' by default if word does not fall in any rule, to correct this let's assign the tags for any such word based purely on transition probability of tags.

So, first we will modify the rule based tagger to output 'NN' instead of 'NOUN' in case word does not satisfy any rules. We also observe that any capitalized word can still be defaulted as 'NOUN' so will add one more rule for that case.

**ref, notes


In [206]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A-Z][a-z].*', 'NOUN'),       # NOUN
    (r'.*', 'NN')                     # default
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [232]:
 # modified Viterbi
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = [] 
        p_transition =[] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            transition_p = tag_p[0]*transition_p
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1] 
        if(pmax==0):
             if state_max == 'NN':
                pmax = max(p_transition)
                state_max = T[p_transition.index(pmax)]                 
        else:
             if state_max != 'X':
                state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [233]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time lapsed: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_2 Algorithm Accuracy: ',accuracy*100)

Time lapsed:  37.195388078689575
Modified Viterbi_2 Algorithm Accuracy:  98.23008849557522


In [234]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')), (('up', 'ADV'), ('up', 'PRT'))]



The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

    Contra:correctly tagged as NOUN
    Honduras:correctly tagged as NOUN
    complaining: correctly tagged as VERB
    Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1

    Sandinista: correctly tagged as NOUN
    Eveready: correctly tagged as NOUN
    *T*-252: correctly tagged as 'X'
    drew: correctly tagged as VERB

Evaluate vanilla Viterbi and modified Viterbi on entire test data

In [244]:
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_run_base = [tup for sent in test_set for tup in sent]

In [245]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time lapsed: ", difference)
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Time lapsed:  1751.204689025879
Modified Viterbi Algorithm Accuracy:  90.89491128875025


In [246]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start
print("Time lapsed: ", difference)
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi1 Algorithm Accuracy: ',accuracy*100)

Time lapsed:  1721.1713027954102
Modified Viterbi1 Algorithm Accuracy:  94.46285825697018


In [ ]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start
print("Time lapsed: ", difference)
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi2 Algorithm Accuracy: ',accuracy*100)

In [ ]:
f = open('Test_sentences.txt')
text = f.read()
sample_test_sent = text.splitlines()
f.close()
sample_test_sent = test_sentences[:-3]
sample_test_sent


In [ ]:
# list of untagged words
sample_test_words = [word for sent in sample_test_sent for word in sent.split()]

In [ ]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi(sample_test_words)
end = time.time()
difference = end-start

print("Time lapsed: ", difference)

In [ ]:
sample_tagged_seq



We can see that several words have been misclassified by vanilla Viterbi POS tagger, for example:

    Android as NUM
    Google as NUM
    OS as NUM



In [ ]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi_2(sample_test_words)
end = time.time()
difference = end-start

print("Time lapsed: ", difference)

In [ ]:
sample_tagged_seq



All these cases were correctly POS tagged by Viterbi_2:

    Android as NOUN
    Google as NOUN
    OS as NOUN




## Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

The accuracy of vanilla Viterbi Algorithm: 91.52%

The accuracy of modified Viterbi Algorithm: 95.44%

## List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

The following cases were incorrectly tagged which got corrected by modified Viterbi Algorithm:

    Contra:correctly tagged as NOUN
    Honduras:correctly tagged as NOUN
    complaining: correctly tagged as VERB
    Bucking: correctly tagged as VERB
    Sandinista: correctly tagged as NOUN
    Eveready: correctly tagged as NOUN
    *T*-252: correctly tagged as 'X'
    drew: correctly tagged as VERB

